In [1]:
# Importing the required libraries
from IPython.display import display
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch.autograd import Variable
from torch_geometric.nn import GCNConv
from torch.utils.data import TensorDataset, DataLoader
import os
import glob
from sklearn.preprocessing import MinMaxScaler


def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

In [2]:
class Net(torch.nn.Module):
    def __init__(self, d_feat, edge_indexs, nnode):
        super(Net, self).__init__()
        self.edge_index = edge_indexs
        self.nnode = nnode
        self.conv1 = GCNConv(d_feat, 64, cached=True,
                             normalize= True)
        self.conv2 = GCNConv(64, 64, cached=True,
                             normalize=True)
        self.conv3 = GCNConv(64, 8, cached=True,
                             normalize=True)
        self.linears = torch.nn.ModuleList([torch.nn.Linear(8, 1) for _ in range(nnode)])


    def forward(self, x):
        x, edge_index, edge_weight = x, self.edge_index, None
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        
        out = [self.linears[i](x[i, :]) for i in range(self.nnode)]

        return out

## Processing the file to input into the model

In [3]:
# Specifying the path of file
path= r"F:\GAMES Research\GAT\GAT\DOE_Q1\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [4]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,5946.0,0.521164,-6.503,0.0,0.0
1,5726.0,0.499457,-6.458,0.0,0.0
2,5347.0,0.462062,-6.253,0.0,0.0
3,5249.0,0.452393,-5.908,0.0,0.0
4,5309.0,0.458313,-5.516,0.0,0.0
...,...,...,...,...,...
43819,6748.0,0.600296,-1.386,0.0,0.0
43820,6429.0,0.568821,-1.661,0.0,0.0
43821,6492.0,0.575037,-1.986,0.0,0.0
43822,6206.0,0.546818,-2.184,0.0,0.0


## Processing the load data into train-test split

In [5]:
# Fetching and concatenating the data
Load_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Demand"]) for fileName in fileList], axis= 1)
Load_DS.columns = [i.removesuffix('.csv') for i in fileList]
Load_DS = Load_DS.add_prefix("LOAD_")

Load_DS

,LOAD_AT,LOAD_BE,LOAD_BG,LOAD_CH,LOAD_CZ,LOAD_DK,LOAD_ES,LOAD_FR,LOAD_GR,LOAD_IT,LOAD_NL,LOAD_PT,LOAD_SI,LOAD_SK
0,5946.0,9484.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,9401.0,0.0,0.00,0.0
1,5726.0,9152.0,4585.0,7640.09,5957.57,3100.02,22684.0,69773.0,5226.83,21534.0,8957.0,5123.9,1045.47,2724.2
2,5347.0,8799.0,4440.0,7391.37,5878.87,2980.39,21236.0,66417.0,4987.34,20219.0,8462.0,4771.1,1004.79,2622.2
3,5249.0,8567.0,4418.0,7332.50,5827.80,2933.49,20222.0,64182.0,4879.48,19470.0,8156.0,4443.5,983.79,2614.2
4,5309.0,8487.0,4393.0,7002.01,5730.89,2941.53,19871.0,63859.0,4909.53,19485.0,8126.0,4234.9,998.67,2622.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,6748.0,9369.0,4760.0,6911.08,6255.85,3785.27,29347.0,64771.0,7530.86,29746.0,12799.0,6788.8,1367.13,3079.5
43820,6429.0,9105.0,4613.0,6638.85,6191.86,3582.21,27593.0,62259.0,6585.61,27140.0,12065.0,6193.0,1261.40,2985.5
43821,6492.0,9346.0,4558.0,6412.14,6070.38,3438.26,25226.0,63774.0,6108.25,25336.0,11641.0,5706.9,1192.50,2964.5
43822,6206.0,9392.0,4410.0,6451.50,5822.73,3309.25,23896.0,65865.0,5657.63,23875.0,11418.0,5484.7,1133.98,2979.5


In [6]:
# Creating 1-day lag loads
Load_lag_1 = Load_DS.shift(24).fillna(0)
Load_lag_1.columns = [i.removeprefix('LOAD_') for i in Load_lag_1.columns]
Load_lag_1 = Load_lag_1.add_prefix("LAG1_")
Load_lag_1

,LAG1_AT,LAG1_BE,LAG1_BG,LAG1_CH,LAG1_CZ,LAG1_DK,LAG1_ES,LAG1_FR,LAG1_GR,LAG1_IT,LAG1_NL,LAG1_PT,LAG1_SI,LAG1_SK
0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
4,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,7397.0,9764.0,5482.0,7334.99,7387.11,4286.58,31257.0,68576.0,7449.33,33480.0,14326.0,7024.8,1510.91,3460.1
43820,6921.0,9254.0,5335.0,7028.54,7085.32,4022.56,31042.0,65849.0,6821.04,31089.0,13601.0,6924.9,1423.19,3301.5
43821,6811.0,9405.0,5043.0,6769.67,6743.37,3753.80,29247.0,66976.0,6274.54,28053.0,12786.0,6646.3,1322.65,3160.5
43822,6348.0,9480.0,4651.0,6855.91,6307.43,3469.41,26678.0,68149.0,5589.98,24968.0,12082.0,6317.3,1196.55,3001.5


In [7]:
# Creating 7-day lag loads
Load_lag_7 = Load_DS.shift(24*7).fillna(0)
Load_lag_7.columns = [i.removeprefix('LOAD_') for i in Load_lag_7.columns]
Load_lag_7 = Load_lag_7.add_prefix("LAG7_")
Load_lag_7

,LAG7_AT,LAG7_BE,LAG7_BG,LAG7_CH,LAG7_CZ,LAG7_DK,LAG7_ES,LAG7_FR,LAG7_GR,LAG7_IT,LAG7_NL,LAG7_PT,LAG7_SI,LAG7_SK
0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
1,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
2,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
3,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
4,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,6158.0,8654.0,4146.0,6337.79,5715.00,3510.73,26307.0,52597.0,6303.19,28134.0,13265.0,6030.4,1256.83,2819.5
43820,5957.0,8253.0,4108.0,6145.57,5604.65,3358.03,24646.0,50947.0,5662.10,25691.0,12680.0,5400.7,1202.55,2796.5
43821,6075.0,8638.0,3964.0,6205.19,5654.16,3243.88,22481.0,52867.0,5237.44,23874.0,12073.0,4945.0,1155.58,2729.5
43822,5748.0,8691.0,3734.0,6645.40,5437.48,3194.35,21518.0,55237.0,4764.68,22415.0,11622.0,4814.1,1074.09,2640.5


In [8]:
# Setting the train-test split
ratio = 0.2
Num_test, Num_train = int(len(Load_DS) * ratio), len(Load_DS) - int(len(Load_DS) * ratio)

In [9]:
# Scaling the load data wrt WEST
mmScaler_load = MinMaxScaler()

# Splitting the data into train and test [LOAD]
Load_train, Load_test = Load_DS[:Num_train], Load_DS[Num_train:]
print("Raw load - Train: ")
display(Load_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 1]
Load_Lag1_train, Load_Lag1_test = Load_lag_1[:Num_train], Load_lag_1[Num_train:]
print("Raw lag 1 load - Train: ")
display(Load_Lag1_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 7]
Load_Lag7_train, Load_Lag7_test = Load_lag_7[:Num_train], Load_lag_7[Num_train:]
print("Raw lag 7 load - Train: ")
display(Load_Lag7_train.head(2))
print("\n")

Raw load - Train: 


,LOAD_AT,LOAD_BE,LOAD_BG,LOAD_CH,LOAD_CZ,LOAD_DK,LOAD_ES,LOAD_FR,LOAD_GR,LOAD_IT,LOAD_NL,LOAD_PT,LOAD_SI,LOAD_SK
0,5946.0,9484.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,9401.0,0.0,0.00,0.0
1,5726.0,9152.0,4585.0,7640.09,5957.57,3100.02,22684.0,69773.0,5226.83,21534.0,8957.0,5123.9,1045.47,2724.2




Raw lag 1 load - Train: 


,LAG1_AT,LAG1_BE,LAG1_BG,LAG1_CH,LAG1_CZ,LAG1_DK,LAG1_ES,LAG1_FR,LAG1_GR,LAG1_IT,LAG1_NL,LAG1_PT,LAG1_SI,LAG1_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0




Raw lag 7 load - Train: 


,LAG7_AT,LAG7_BE,LAG7_BG,LAG7_CH,LAG7_CZ,LAG7_DK,LAG7_ES,LAG7_FR,LAG7_GR,LAG7_IT,LAG7_NL,LAG7_PT,LAG7_SI,LAG7_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Scaling the data using mix-max scaler [TRAINING]
Load_train = mmScaler_load.fit_transform(Load_train)
print("Scaled load - Train: ")
display(Load_train)

Load_Lag1_train = mmScaler_load.transform(Load_Lag1_train)
print("Scaled lag 1 load - Train: ")
display(Load_Lag1_train)

Load_Lag7_train = mmScaler_load.transform(Load_Lag7_train)
print("Scaled lag 7 load - Train: ")
display(Load_Lag7_train)

Scaled load - Train: 


array([[0.52116428, 0.46443262, 0.07103165, ..., 0.        , 0.        ,
        0.0132673 ],
       [0.49945733, 0.4219557 , 0.6249094 , ..., 0.58678225, 0.43521537,
        0.16693555],
       [0.46206216, 0.37679199, 0.60739309, ..., 0.54638006, 0.41828082,
        0.16118187],
       ...,
       [0.69669462, 0.55296884, 0.77675767, ..., 0.68312682, 0.64279262,
        0.1993874 ],
       [0.64893932, 0.48643873, 0.70053153, ..., 0.75764412, 0.60127217,
        0.19199788],
       [0.60996547, 0.42656163, 0.65076105, ..., 0.75113946, 0.55880259,
        0.18771647]])

Scaled lag 1 load - Train: 


array([[-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       [-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       [-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       ...,
       [ 0.69541194,  0.55539975,  0.75924136, ...,  0.63924326,
         0.63517457,  0.20296935],
       [ 0.66097681,  0.53863928,  0.73169848, ...,  0.71108083,
         0.63847989,  0.2014858 ],
       [ 0.62289097,  0.49040495,  0.70379319, ...,  0.74630677,
         0.6269196 ,  0.19878383]])

Scaled lag 7 load - Train: 


array([[-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       [-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       [-0.06551554, -0.74897422,  0.07103165, ...,  0.        ,
         0.        ,  0.0132673 ],
       ...,
       [ 0.65545141,  0.56781017,  0.69050495, ...,  0.69172717,
         0.62028815,  0.1867688 ],
       [ 0.60157869,  0.51356255,  0.64351293, ...,  0.72966721,
         0.58624422,  0.18129717],
       [ 0.57335964,  0.47300477,  0.62285576, ...,  0.71292458,
         0.55665455,  0.17966132]])

In [11]:
# Scaling the data using mix-max scaler [TESTING]
Load_test = mmScaler_load.transform(Load_test)
print("Scaled load - Test: ")
display(Load_test)

Load_Lag1_test = mmScaler_load.transform(Load_Lag1_test)
print("Scaled lag 1 load - Test: ")
display(Load_Lag1_test)

Load_Lag7_test = mmScaler_load.transform(Load_Lag7_test)
print("Scaled lag 7 load - Test: ")
display(Load_Lag7_test)

Scaled load - Test: 


array([[0.57316231, 0.3972628 , 0.6363856 , ..., 0.68856646, 0.52224429,
        0.18197407],
       [0.58263444, 0.43027197, 0.62756705, ..., 0.6305742 , 0.50169637,
        0.17892801],
       [0.55510607, 0.44408976, 0.61319159, ..., 0.60429216, 0.48018267,
        0.17655885],
       ...,
       [0.575037  , 0.44677655, 0.62164774, ..., 0.65354664, 0.49642201,
        0.18049053],
       [0.54681796, 0.45266191, 0.60376903, ..., 0.62810059, 0.47206091,
        0.18133666],
       [0.52323631, 0.41133647, 0.58359507, ..., 0.60051304, 0.45236638,
        0.17197283]])

Scaled lag 1 load - Test: 


array([[0.58825851, 0.44050737, 0.69183378, ..., 0.74921555, 0.59574388,
        0.19011947],
       [0.59131722, 0.45317368, 0.66151244, ..., 0.72671263, 0.55861526,
        0.1817146 ],
       [0.5476073 , 0.44434564, 0.6137956 , ..., 0.69164701, 0.50898555,
        0.17339433],
       ...,
       [0.60651209, 0.45432516, 0.68023677, ..., 0.76112549, 0.55060174,
        0.19154661],
       [0.56082881, 0.46392085, 0.63288234, ..., 0.72344884, 0.49810798,
        0.18257765],
       [0.52856438, 0.40353198, 0.60171539, ..., 0.66423124, 0.45998027,
        0.17490608]])

Scaled lag 7 load - Test: 


array([[0.55559941, 0.44805598, 0.61971491, ..., 0.64248414, 0.5286218 ,
        0.17909724],
       [0.57602368, 0.47543568, 0.60896352, ..., 0.59351595, 0.51724884,
        0.17712294],
       [0.54415392, 0.49744179, 0.57755496, ..., 0.57949887, 0.48086121,
        0.17306152],
       ...,
       [0.53389245, 0.35619325, 0.54989128, ..., 0.56629486, 0.48105271,
        0.16723451],
       [0.50162802, 0.3629742 , 0.52210679, ..., 0.55130437, 0.44712949,
        0.16221415],
       [0.47163296, 0.32126494, 0.49504711, ..., 0.53631387, 0.41494636,
        0.15747583]])

In [12]:
## Transformation assigns perfectly.
Load_train[0]

array([0.52116428, 0.46443262, 0.07103165, 0.        , 0.08852283,
       0.        , 0.0857245 , 0.0536087 , 0.12957982, 0.        ,
       0.22816483, 0.        , 0.        , 0.0132673 ])

In [13]:
Load_Lag1_train[24]

array([0.52116428, 0.46443262, 0.07103165, 0.        , 0.08852283,
       0.        , 0.0857245 , 0.0536087 , 0.12957982, 0.        ,
       0.22816483, 0.        , 0.        , 0.0132673 ])

In [14]:
Load_Lag7_train[24*7]

array([0.52116428, 0.46443262, 0.07103165, 0.        , 0.08852283,
       0.        , 0.0857245 , 0.0536087 , 0.12957982, 0.        ,
       0.22816483, 0.        , 0.        , 0.0132673 ])

In [15]:
## Transformation assigns perfectly.
Load_test[5]

array([0.48889985, 0.2844175 , 0.55315294, 0.36154762, 0.55894549,
       0.35681768, 0.56201394, 0.64107659, 0.59208515, 0.39160941,
       0.34795527, 0.53246605, 0.42087845, 0.1643746 ])

In [16]:
Load_Lag1_test[24+5]

array([0.48889985, 0.2844175 , 0.55315294, 0.36154762, 0.55894549,
       0.35681768, 0.56201394, 0.64107659, 0.59208515, 0.39160941,
       0.34795527, 0.53246605, 0.42087845, 0.1643746 ])

In [17]:
Load_Lag7_test[24*7+5]

array([0.48889985, 0.2844175 , 0.55315294, 0.36154762, 0.55894549,
       0.35681768, 0.56201394, 0.64107659, 0.59208515, 0.39160941,
       0.34795527, 0.53246605, 0.42087845, 0.1643746 ])

## Processing the temperature data in train test split

In [18]:
# Fetching and concatenating the data
Temp_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Temperature"]) for fileName in fileList], axis= 1)
Temp_DS.columns = [i.removesuffix('.csv') for i in fileList]
Temp_DS = Temp_DS.add_prefix("TEMP_")

Temp_DS

,TEMP_AT,TEMP_BE,TEMP_BG,TEMP_CH,TEMP_CZ,TEMP_DK,TEMP_ES,TEMP_FR,TEMP_GR,TEMP_IT,TEMP_NL,TEMP_PT,TEMP_SI,TEMP_SK
0,-6.503,-0.046,-13.350,-9.152,-2.372,3.983,3.438,-1.670,1.248,-1.123,1.869,5.584,-11.261,-8.272
1,-6.458,-0.302,-13.594,-8.946,-2.126,4.273,3.217,-1.768,1.192,-1.111,1.625,5.219,-10.975,-8.114
2,-6.253,-0.541,-13.781,-8.676,-1.839,4.563,3.103,-1.841,1.129,-1.036,1.448,5.034,-10.772,-7.898
3,-5.908,-0.794,-14.042,-8.373,-1.521,4.674,3.051,-1.908,1.040,-0.955,1.250,4.708,-10.667,-7.670
4,-5.516,-1.053,-14.340,-8.140,-1.223,4.635,2.982,-1.962,0.850,-0.905,1.102,4.353,-10.467,-7.399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,-1.386,1.684,-1.351,-0.174,0.048,1.961,8.295,2.048,4.503,3.951,2.909,9.646,0.053,-0.766
43820,-1.661,1.718,-1.009,-0.627,-0.163,1.874,7.805,1.995,4.342,3.825,2.635,9.172,-0.207,-0.742
43821,-1.986,1.726,-0.986,-0.723,-0.274,1.957,7.342,2.107,4.219,3.764,2.210,8.705,-0.203,-0.764
43822,-2.184,1.569,-1.270,-0.659,-0.311,2.213,6.959,2.280,4.143,3.752,1.785,8.218,-0.032,-0.867


In [19]:
# Scaling the temperature data for each individual region
mmScaler_temp = MinMaxScaler()

# Splitting the data into train and test
Temp_train, Temp_test = Temp_DS[:Num_train], Temp_DS[Num_train:]
print("Raw temperature - Train: ")
display(Temp_train.head(5))
print("Raw temperature - Test: ")
display(Temp_test.head(5))
print("\n")

# Scaling the data using mix-max scaler
Temp_train = mmScaler_temp.fit_transform(Temp_train)
print("Scaled temperature - Train: ")
display(Temp_train)

Temp_test = mmScaler_temp.transform(Temp_test)
print("Scaled temperature - Test: ")
display(Temp_test)

Raw temperature - Train: 


,TEMP_AT,TEMP_BE,TEMP_BG,TEMP_CH,TEMP_CZ,TEMP_DK,TEMP_ES,TEMP_FR,TEMP_GR,TEMP_IT,TEMP_NL,TEMP_PT,TEMP_SI,TEMP_SK
0,-6.503,-0.046,-13.350,-9.152,-2.372,3.983,3.438,-1.670,1.248,-1.123,1.869,5.584,-11.261,-8.272
1,-6.458,-0.302,-13.594,-8.946,-2.126,4.273,3.217,-1.768,1.192,-1.111,1.625,5.219,-10.975,-8.114
2,-6.253,-0.541,-13.781,-8.676,-1.839,4.563,3.103,-1.841,1.129,-1.036,1.448,5.034,-10.772,-7.898
3,-5.908,-0.794,-14.042,-8.373,-1.521,4.674,3.051,-1.908,1.040,-0.955,1.250,4.708,-10.667,-7.670
4,-5.516,-1.053,-14.340,-8.140,-1.223,4.635,2.982,-1.962,0.850,-0.905,1.102,4.353,-10.467,-7.399


Raw temperature - Test: 


,TEMP_AT,TEMP_BE,TEMP_BG,TEMP_CH,TEMP_CZ,TEMP_DK,TEMP_ES,TEMP_FR,TEMP_GR,TEMP_IT,TEMP_NL,TEMP_PT,TEMP_SI,TEMP_SK
35060,-1.408,6.348,-1.479,0.564,-0.369,6.400,8.468,5.042,6.057,4.432,7.411,10.045,-1.412,-4.234
35061,-1.614,6.135,-1.614,0.304,-0.333,6.461,7.961,4.897,5.867,4.121,7.279,9.417,-1.380,-4.585
35062,-1.688,5.823,-1.792,0.046,-0.072,6.601,7.622,4.805,5.810,3.833,7.216,8.894,-1.295,-4.733
35063,-1.666,5.753,-1.982,-0.070,0.219,6.795,7.275,4.697,5.836,3.553,7.232,8.347,-1.224,-4.664
35064,-1.524,5.750,-2.142,-0.278,0.371,7.009,6.994,4.569,5.765,3.295,7.272,7.831,-1.218,-4.549




Scaled temperature - Train: 


array([[0.22085731, 0.21920426, 0.05045186, ..., 0.14340302, 0.16690025,
        0.19659711],
       [0.22175324, 0.21338065, 0.04600612, ..., 0.13329641, 0.17197396,
        0.19950966],
       [0.22583471, 0.20794377, 0.04259894, ..., 0.12817389, 0.17557523,
        0.20349137],
       ...,
       [0.33264977, 0.36634136, 0.28015451, ..., 0.32241451, 0.34934095,
        0.29623212],
       [0.32998188, 0.36645511, 0.27399606, ..., 0.30242282, 0.34604126,
        0.28799218],
       [0.32633842, 0.36681908, 0.26975075, ..., 0.28445244, 0.34307864,
        0.27943887]])

Scaled temperature - Test: 


array([[0.32229677, 0.36465798, 0.26674441, ..., 0.2669251 , 0.34162394,
        0.27103303],
       [0.31819539, 0.35981255, 0.26428467, ..., 0.2495362 , 0.34219163,
        0.26456275],
       [0.31672208, 0.35271503, 0.26104147, ..., 0.23505469, 0.34369955,
        0.26183454],
       ...,
       [0.31078902, 0.25951455, 0.27572699, ..., 0.2298214 , 0.3630719 ,
        0.33499853],
       [0.30684692, 0.25594304, 0.27055244, ..., 0.2163367 , 0.36610548,
        0.33309984],
       [0.30511478, 0.24791283, 0.26042198, ..., 0.20296276, 0.36294772,
        0.3318279 ]])

## Processing the DNI data in train test split

In [20]:
# Fetching and concatenating the data
DNI_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["DNI"]) for fileName in fileList], axis= 1)
DNI_DS.columns = [i.removesuffix('.csv') for i in fileList]
DNI_DS = DNI_DS.add_prefix("DNI_")

DNI_DS

,DNI_AT,DNI_BE,DNI_BG,DNI_CH,DNI_CZ,DNI_DK,DNI_ES,DNI_FR,DNI_GR,DNI_IT,DNI_NL,DNI_PT,DNI_SI,DNI_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Scaling the temperature data for each individual region
mmScaler_DNI = MinMaxScaler()

# Splitting the data into train and test
DNI_train, DNI_test = DNI_DS[:Num_train], DNI_DS[Num_train:]
print("Raw DNI - Train: ")
display(DNI_train.head(2))
print("\n")

# Scaling the data using mix-max scaler
DNI_train = mmScaler_DNI.fit_transform(DNI_train)
print("Scaled DNI - Train: ")
display(DNI_train)

DNI_test = mmScaler_DNI.transform(DNI_test)
print("Scaled DNI - Test: ")
display(DNI_test)

Raw DNI - Train: 


,DNI_AT,DNI_BE,DNI_BG,DNI_CH,DNI_CZ,DNI_DK,DNI_ES,DNI_FR,DNI_GR,DNI_IT,DNI_NL,DNI_PT,DNI_SI,DNI_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0




Scaled DNI - Train: 


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.46556861e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.14076630e-05, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

Scaled DNI - Test: 


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Processing the DHI data in train test split

In [22]:
# Fetching and concatenating the data
DHI_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["DHI"]) for fileName in fileList], axis= 1)
DHI_DS.columns = [i.removesuffix('.csv') for i in fileList]
DHI_DS = DHI_DS.add_prefix("DHI_")

DHI_DS

,DHI_AT,DHI_BE,DHI_BG,DHI_CH,DHI_CZ,DHI_DK,DHI_ES,DHI_FR,DHI_GR,DHI_IT,DHI_NL,DHI_PT,DHI_SI,DHI_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43822,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Scaling the temperature data for each individual region
mmScaler_DHI = MinMaxScaler()

# Splitting the data into train and test
DHI_train, DHI_test = DHI_DS[:Num_train], DHI_DS[Num_train:]
print("Raw DHI - Train: ")
display(DHI_train.head(2))
print("\n")

# Scaling the data using mix-max scaler
DHI_train = mmScaler_DHI.fit_transform(DHI_train)
print("Scaled DHI - Train: ")
display(DHI_train)

DHI_test = mmScaler_DHI.transform(DHI_test)
print("Scaled DHI - Test: ")
display(DHI_test)

Raw DHI - Train: 


,DHI_AT,DHI_BE,DHI_BG,DHI_CH,DHI_CZ,DHI_DK,DHI_ES,DHI_FR,DHI_GR,DHI_IT,DHI_NL,DHI_PT,DHI_SI,DHI_SK
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0




Scaled DHI - Train: 


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.01013913, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00040828, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Scaled DHI - Test: 


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Preparing the training and testing split for model input

In [24]:
# X_train = np.array([[[i, j, k, y, z] for i,j,k,y,z in zip(Temp_train[m], Load_Lag1_train[m], Load_Lag7_train[m], DNI_train[m], DHI_train[m])] for m in range(len(Temp_train))])
# print(X_train.shape)
# print(X_train[0])

X_train = np.array([[[i, j, k] for i,j,k in zip(Temp_train[m], Load_Lag1_train[m], Load_Lag7_train[m])] for m in range(len(Temp_train))])
print(X_train.shape)
print(X_train[0])

(35060, 14, 3)
[[ 0.22085731 -0.06551554 -0.06551554]
 [ 0.21920426 -0.74897422 -0.74897422]
 [ 0.05045186  0.07103165  0.07103165]
 [ 0.15311557  0.          0.        ]
 [ 0.26543821  0.08852283  0.08852283]
 [ 0.32740244  0.          0.        ]
 [ 0.11631718  0.0857245   0.0857245 ]
 [ 0.1597617   0.0536087   0.0536087 ]
 [ 0.10369437  0.12957982  0.12957982]
 [ 0.07975194  0.          0.        ]
 [ 0.24515136 -0.50692001 -0.50692001]
 [ 0.14340302  0.          0.        ]
 [ 0.16690025  0.          0.        ]
 [ 0.19659711  0.0132673   0.0132673 ]]


In [25]:
# X_test = np.array([[[i, j, k, y, z] for i,j,k,y,z in zip(Temp_test[m], Load_Lag1_test[m], Load_Lag7_test[m], DNI_test[m], DHI_test[m])] for m in range(len(Temp_test))])
# print(X_test.shape)
# print(X_test[0])

X_test = np.array([[[i, j, k] for i,j,k in zip(Temp_test[m], Load_Lag1_test[m], Load_Lag7_test[m])] for m in range(len(Temp_test))])
print(X_test.shape)
print(X_test[0])

(8764, 14, 3)
[[0.32229677 0.58825851 0.55559941]
 [0.36465798 0.44050737 0.44805598]
 [0.26674441 0.69183378 0.61971491]
 [0.3609489  0.36885214 0.37782588]
 [0.30356735 0.64076701 0.59200025]
 [0.39610039 0.43197083 0.40622127]
 [0.25602    0.7605703  0.67967283]
 [0.32637458 0.68020432 0.58357418]
 [0.21585502 0.74164827 0.68276505]
 [0.21379793 0.53538082 0.50470475]
 [0.38277583 0.50973493 0.51090781]
 [0.2669251  0.74921555 0.64248414]
 [0.34162394 0.59574388 0.5286218 ]
 [0.27103303 0.19011947 0.17909724]]


In [26]:
# Setting up the batch and node parameters
num_batch, num_node = Load_DS.shape

number_feat = X_train.shape[2]

print("No. of batches: ", num_batch)
print("No. of nodes: " , num_node)
print("No. of features: ", number_feat)

No. of batches:  43824
No. of nodes:  14
No. of features:  3


In [27]:
edge_index = torch.LongTensor([np.repeat(range(num_node), num_node-1).tolist(),
                               [j for i in range(num_node) for j in range(num_node) if i != j]])

X_train, X_test = Variable(torch.FloatTensor(X_train)), torch.FloatTensor(X_test)
Y_train, Y_test = Variable(torch.FloatTensor(Load_train)), Load_test

In [28]:
edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
          6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
          8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,
          9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11,
         11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
         12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
         13, 13],
        [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  2,  3,  4,  5,
          6,  7,  8,  9, 10, 11, 12, 13,  0,  1,  3,  4,  5,  6,  7,  8,  9, 10,
         1

## DEFINING THE MODEL

In [29]:
torch.manual_seed(666)

model = Net(number_feat, edge_index, num_node)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0),
    dict(params=model.conv3.parameters(), weight_decay=0)
], lr=0.01) 

In [30]:
Epochs = 100

dataset = TensorDataset(X_train, Y_train)    
loader = DataLoader(dataset, batch_size = 64, shuffle=False)  

In [31]:
model.train()

for epoch in range(Epochs):
    loss = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        loss_train = 0
        for i in range(x.size(0)):
            output = model(x[i])
            loss_train += F.l1_loss(output[0][0], y[i,0]) + F.l1_loss(output[1][0], y[i,1]) + F.l1_loss(output[2][0], y[i,2]) + F.l1_loss(output[3][0], y[i,3]) + F.l1_loss(output[4][0], y[i,4]) + F.l1_loss(output[5][0], y[i,5]) + F.l1_loss(output[6][0], y[i,6]) + F.l1_loss(output[7][0], y[i,7]) + F.l1_loss(output[8][0], y[i,8]) + F.l1_loss(output[9][0], y[i,9]) + F.l1_loss(output[10][0], y[i,10]) + F.l1_loss(output[11][0], y[i,11]) + F.l1_loss(output[12][0], y[i,12]) + F.l1_loss(output[13][0], y[i,13])
        loss += loss_train/i
        loss_train.backward()
        optimizer.step()
    loss = loss/step
    print( "Epoch {}: the train loss = {:.4f}".format(epoch+1, loss))

Epoch 1: the train loss = 5.1492
Epoch 2: the train loss = 5.1046
Epoch 3: the train loss = 5.1000
Epoch 4: the train loss = 5.1010
Epoch 5: the train loss = 5.0965
Epoch 6: the train loss = 5.0962
Epoch 7: the train loss = 5.0963
Epoch 8: the train loss = 5.0942
Epoch 9: the train loss = 5.0939
Epoch 10: the train loss = 5.0908
Epoch 11: the train loss = 5.0907
Epoch 12: the train loss = 5.0902
Epoch 13: the train loss = 5.0897
Epoch 14: the train loss = 5.0898
Epoch 15: the train loss = 5.0889
Epoch 16: the train loss = 5.0886
Epoch 17: the train loss = 5.0873
Epoch 18: the train loss = 5.0874
Epoch 19: the train loss = 5.0873
Epoch 20: the train loss = 5.0874
Epoch 21: the train loss = 5.0870
Epoch 22: the train loss = 5.0869
Epoch 23: the train loss = 5.0872
Epoch 24: the train loss = 5.0871
Epoch 25: the train loss = 5.0875
Epoch 26: the train loss = 5.0870
Epoch 27: the train loss = 5.0866
Epoch 28: the train loss = 5.0867
Epoch 29: the train loss = 5.0866
Epoch 30: the train los

KeyboardInterrupt: 

## MAKING PREDICTIONS

In [32]:
predictions = []

# Switching to eval mode
model.eval()

with torch.no_grad():
    for i in range(X_test.size(0)):
        p = model(X_test[i])
        predictions.append(torch.cat([p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7], p[8], p[9], p[10], p[11], p[12], p[13]], dim=-1).tolist())

predictions = np.array(predictions)

In [33]:
predictions

array([[ 0.54164046,  0.16746779,  0.01933688, ..., -0.26967314,
         0.57381296,  0.08540097],
       [ 0.52472961,  0.15342788,  0.02906561, ..., -0.25744322,
         0.55207986,  0.08672515],
       [ 0.49536052,  0.12904494,  0.0459615 , ..., -0.23620351,
         0.51433599,  0.08902478],
       ...,
       [ 0.50269574,  0.13513477,  0.04174164, ..., -0.24150828,
         0.52376276,  0.08845045],
       [ 0.4691669 ,  0.10729824,  0.06103057, ..., -0.21726024,
         0.48067302,  0.09107581],
       [ 0.44623148,  0.08825664,  0.07422522, ..., -0.20067336,
         0.45119745,  0.09287173]])

In [34]:
inversed_predictions = mmScaler_load.inverse_transform(predictions)
inversed_predictions

array([[ 6153.52606708,  7162.91990461,  -427.92931354, ...,
        -2354.83978313,  1378.40775817,  1278.77129452],
       [ 5982.13459218,  7053.18387549,  -347.39488888, ...,
        -2248.04568065,  1326.20071024,  1302.24603578],
       [ 5684.47890529,  6862.60650325,  -207.53070951, ...,
        -2062.57626024,  1235.5327717 ,  1343.01353354],
       ...,
       [ 5758.82131749,  6910.20472513,  -242.46270925, ...,
        -2108.89856132,  1258.17767056,  1332.83182945],
       [ 5419.00657901,  6692.63411733,   -82.78896892, ...,
        -1897.1598809 ,  1154.66791019,  1379.37370569],
       [ 5186.55609453,  6543.80479818,    26.43634719, ...,
        -1752.31988496,  1083.86199135,  1411.21137637]])

In [35]:
inversed_Y_test = mmScaler_load.inverse_transform(Y_test)
inversed_Y_test

array([[6473.  , 8959.  , 4680.  , ..., 6012.7 , 1254.53, 2990.8 ],
       [6569.  , 9217.  , 4607.  , ..., 5506.3 , 1205.17, 2936.8 ],
       [6290.  , 9325.  , 4488.  , ..., 5276.8 , 1153.49, 2894.8 ],
       ...,
       [6492.  , 9346.  , 4558.  , ..., 5706.9 , 1192.5 , 2964.5 ],
       [6206.  , 9392.  , 4410.  , ..., 5484.7 , 1133.98, 2979.5 ],
       [5967.  , 9069.  , 4243.  , ..., 5243.8 , 1086.67, 2813.5 ]])

In [36]:
MAPE1 = mape(inversed_Y_test[:, 0].reshape(-1), inversed_predictions[:, 0].reshape(-1))
MAPE2 = mape(inversed_Y_test[:, 1].reshape(-1), inversed_predictions[:, 1].reshape(-1))
MAPE3 = mape(inversed_Y_test[:, 2].reshape(-1), inversed_predictions[:, 2].reshape(-1))
MAPE4 = mape(inversed_Y_test[:, 3].reshape(-1), inversed_predictions[:, 3].reshape(-1))
MAPE5 = mape(inversed_Y_test[:, 4].reshape(-1), inversed_predictions[:, 4].reshape(-1))
MAPE6 = mape(inversed_Y_test[:, 5].reshape(-1), inversed_predictions[:, 5].reshape(-1))
MAPE7 = mape(inversed_Y_test[:, 6].reshape(-1), inversed_predictions[:, 6].reshape(-1))
MAPE8 = mape(inversed_Y_test[:, 7].reshape(-1), inversed_predictions[:, 7].reshape(-1))
MAPE9 = mape(inversed_Y_test[:, 8].reshape(-1), inversed_predictions[:, 8].reshape(-1))
MAPE10 = mape(inversed_Y_test[:, 9].reshape(-1), inversed_predictions[:, 9].reshape(-1))
MAPE11 = mape(inversed_Y_test[:, 10].reshape(-1), inversed_predictions[:, 10].reshape(-1))
MAPE12 = mape(inversed_Y_test[:, 11].reshape(-1), inversed_predictions[:, 11].reshape(-1))
MAPE13 = mape(inversed_Y_test[:, 12].reshape(-1), inversed_predictions[:, 12].reshape(-1))
MAPE14 = mape(inversed_Y_test[:, 13].reshape(-1), inversed_predictions[:, 13].reshape(-1))


print( "The test mape is {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8, MAPE9, MAPE10, MAPE11, MAPE12, MAPE13, MAPE14))

The test mape is 10.583, 16.621, 62.571, 9.042, 8.803, 48.492, 71.121, 22.474, 11.446, 136.408, 21.429, 91.960, 6.510, 44.202


#### AVERAGE MAPE

In [37]:
meanMAPE = np.mean([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8, MAPE9, MAPE10, MAPE11, MAPE12, MAPE13, MAPE14])
meanMAPE

40.11867970801477

#### WEIGHTED MAPE

In [38]:
# Scaled MAPE with max values
weightMax = pd.Series(Load_DS.max() / sum(Load_DS.max())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8, MAPE9, MAPE10, MAPE11, MAPE12, MAPE13, MAPE14])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the max load of a region: ", weightedMAPE)

0     0.033970
1     0.043001
2     0.024190
3     0.058288
4     0.034267
5     0.027065
6     0.128695
7     0.296655
8     0.029690
9     0.173505
10    0.060623
11    0.027469
12    0.007556
13    0.055026
dtype: float64


0      10.582983
1      16.620893
2      62.570506
3       9.042326
4       8.802997
5      48.491594
6      71.120771
7      22.473887
8      11.446331
9     136.408427
10     21.428553
11     91.959918
12      6.510192
13     44.202139
dtype: float64


Weighted using the max load of a region:  50.862759438861794


In [39]:
# Scaled MAPE with average values
weightMax = pd.Series(Load_DS.mean() / sum(Load_DS.mean())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8, MAPE9, MAPE10, MAPE11, MAPE12, MAPE13, MAPE14])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the mean load of a region: ", weightedMAPE)

0     0.039315
1     0.053698
2     0.023466
3     0.037409
4     0.040494
5     0.020602
6     0.152753
7     0.297148
8     0.030527
9     0.176638
10    0.070141
11    0.031399
12    0.008118
13    0.018293
dtype: float64


0      10.582983
1      16.620893
2      62.570506
3       9.042326
4       8.802997
5      48.491594
6      71.120771
7      22.473887
8      11.446331
9     136.408427
10     21.428553
11     91.959918
12      6.510192
13     44.202139
dtype: float64


Weighted using the mean load of a region:  51.70880765043461
